<a href="https://colab.research.google.com/github/adrian-sastrawiria/reckoner-lp-optimizer/blob/main/cost-simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Zappa Cost *Simulation* v0**
last modified : 2020-08-28 14
:00
last modified by : adrian.m@go-jek.com

## 0. Installing pulp and importing packages needed

In [ ]:
import sys
!{sys.executable} -m pip install pulp
!{sys.executable} -m pip install gspread

import gspread
import os
import pandas as pd
import numpy as np
import pulp as p
import datetime 

pd.set_option('display.float_format', lambda x: '%.2f' % x)

### 0.1. Mounting GDrive for Google Colab
Follow the prompts, enter the authorization codes for GDrive & Google SDK

In [ ]:
from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

drive.mount('/content/gdrive')
root_path = '/content/gdrive/Shared drives/Zappa Solver'
os.chdir(root_path)

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

### 0.2. Defining Functions needed by the script

In [ ]:
# Defining Function gsdf to take data from Google Sheet and convert it to DataFrame
def gsdf(x,y):
  ws = gc.open(x).worksheet(y)
  rows = ws.get_all_values()
  df = pd.DataFrame.from_records(rows)
  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header
  return df

# Defining Function zappa_output to prepare linear programming output sheets 
def zappa_output(x):
  ou = fin.pivot(index='sortv', columns='sortt', values=[x])
  ou.columns = ou.columns.droplevel(0) 
  ou.columns.name = None
  ou = ou.reset_index()
  ou = ou.merge(dt_pop[['index',seg_column_pop]], left_on = 'sortv', right_on = 'index', how='inner')
  cols = ou.columns.tolist()
  cols = cols[-1:] + cols[:-1]
  ou = ou[cols].drop(columns=['sortv', 'index']).add_prefix('Treatment ')
  return ou

def weird_division(n, d):
    return n / d if d else 0

## 1. Input Preparation

In [ ]:
filename_all = 'lazarus_sensi_input_avg_22feb_1mar_t5' # put the filename that contains cost input sheet (CPU & Inc Unit per User metrics) & population input sheet
city = 'All'    # input the city you are optimizing
inp_budget = 139771 # input the budget
pop_pctg = 1   # input the proportion of population you want to assign to the model

In [ ]:
# Def Function for Simulation

def simbud(bpctg):
  budget = inp_budget * (bpctg/100)
  # Loading CPU and IRPU data
  dt = gsdf(filename_all,'cost_input')
  
  seg_column = dt.columns[0]
  trt_column = dt.columns[1]
  cpu_column = dt.columns[2]
  inc_column = dt.columns[3]
  
  
  dt[cpu_column] = dt[cpu_column].astype('float64')
  dt[inc_column] = dt[inc_column].astype('float64')
  
  dt['vars'] = dt[seg_column] + dt[trt_column]
  dt['vars'] = dt['vars'].str.replace(" ","")
  dt['vars'] = dt['vars'].str.replace("'","")
  dt['vars'] = dt['vars'].str.replace("-","")
  dt['SegmentCod'] = dt[seg_column].str.replace(" ", "")
  dt['SegmentCod'] = dt['SegmentCod'].str.replace("'", "")
  dt['SegmentCod'] = dt['SegmentCod'].str.replace("-", "")
  
  # Loading Population Size per Segment data
  dt_pop = gsdf(filename_all, 'pop_input')
  
  
  seg_column_pop = dt_pop.columns[0]
  pop_column_pop = dt_pop.columns[1]
  
  dt_pop[pop_column_pop] = dt_pop[pop_column_pop].astype('int64')
  
  dt_pop[pop_column_pop] = dt_pop[pop_column_pop] * pop_pctg
  dt_pop[pop_column_pop] = dt_pop[pop_column_pop].round()
  dt_pop['SegmentCod'] = dt_pop[seg_column_pop].str.replace(" ", "")
  dt_pop['SegmentCod'] = dt_pop['SegmentCod'].str.replace("'", "")
  dt_pop['SegmentCod'] = dt_pop['SegmentCod'].str.replace("-", "")
  dt_pop.reset_index(level=0, inplace=True)
  
  # Setting Up Problem
  lp = p.LpProblem('Problem', p.LpMaximize)
  
  # Preparing Objective Function
  fob = ''
  for index, row in dt.iterrows():
      fob = fob + str(row[inc_column]) + ' * ' + str(row['vars']) + ' + '
      
  fob = fob.rstrip(' + ')
  
  # Preparing Cost Constraint
  constraint_cost = ''
  
  for index, row in dt.iterrows():
      constraint_cost = constraint_cost + str(row[cpu_column]) + ' * ' + str(row['vars']) + ' + '
      
  constraint_cost = constraint_cost.rstrip(' + ') + ' <= ' + str(budget)
  
  # Setting up Variables
  for index, row in dt.iterrows():
      globals()[row['vars']] = p.LpVariable(row['vars'], lowBound = 0, cat='Integer')
      
  # Setting Objective Function
  lp += eval(fob)
  
  #Setting Cost Constraint
  lp += eval(constraint_cost)
  
  # Setting Up Population Constraint
  for index, row in dt_pop.iterrows():
          b = ''
          pop = row[pop_column_pop]
          segname = row[seg_column_pop]
          for index, row in dt.iterrows():
              if row[seg_column_pop] == segname:
                  b = b + str(row['vars']) + ' + '
              else:
                  b = b
          if b != '':
              b = b.rstrip(' + ') + ' <= ' + str(pop)
              globals()[segname.replace(' ','')] = b
              lp += eval(b)
  
  # Setting Up Problem
  lp = p.LpProblem('Problem', p.LpMaximize)
  
  # Preparing Objective Function
  fob = ''
  for index, row in dt.iterrows():
      fob = fob + str(row[inc_column]) + ' * ' + str(row['vars']) + ' + '
      
  fob = fob.rstrip(' + ')
  
  # Preparing Cost Constraint
  constraint_cost = ''
  
  for index, row in dt.iterrows():
      constraint_cost = constraint_cost + str(row[cpu_column]) + ' * ' + str(row['vars']) + ' + '
      
  constraint_cost = constraint_cost.rstrip(' + ') + ' <= ' + str(budget)
  
  # Setting up Variables
  for index, row in dt.iterrows():
      globals()[row['vars']] = p.LpVariable(row['vars'], lowBound = 0, cat='Integer')
      
  # Setting Objective Function
  lp += eval(fob)
  
  #Setting Cost Constraint
  lp += eval(constraint_cost)
  
  # Setting Up Population Constraint
  for index, row in dt_pop.iterrows():
          b = ''
          pop = row[pop_column_pop]
          segname = row[seg_column_pop]
          for index, row in dt.iterrows():
              if row[seg_column_pop] == segname:
                  b = b + str(row['vars']) + ' + '
              else:
                  b = b
          if b != '':
              b = b.rstrip(' + ') + ' <= ' + str(pop)
              globals()[segname.replace(' ','')] = b
              lp += eval(b)
              
  # Solving the Model
  status = lp.solve() 
  
  # Evaluation Metrics
  cst = str(lp.constraints['_C1']).replace(' <= ' + str(budget),'').rstrip('.0')
  cst = cst.replace('*','*p.value(').replace(' +',') +') + ')'
  absorbed_budget = eval(cst) 
  
  st = pd.DataFrame([[bpctg/100
                     ,  budget
                     ,  absorbed_budget
                     ,  round(weird_division(budget,p.value(lp.objective)),2)
                     ,  round(p.value(lp.objective),0)]
                    ], columns = ['Budget Pctg'
                                 ,  'Budget'
                                 ,  'Absorbed Budget'
                                 ,  'Optimized Cost per Incremental Unit'
                                 ,  'Incremental Unit'
                                 ])
  st.astype({'Budget Pctg' : 'float'
            ,  'Budget': 'float'
            , 'Absorbed Budget' : 'float'
            , 'Optimized Cost per Incremental Unit' : 'float'
            , 'Incremental Unit' : 'float'})
  
  return st

In [ ]:
st_fin = pd.DataFrame([[0, 0,  0,  0,  0]], columns = ['Budget Pctg' ,'Budget' ,  'Absorbed Budget' ,  'Optimized Cost per Incremental Unit',  'Incremental Unit'])

bpctg = 100

while bpctg > 0:
  st = simbud(bpctg)
  st_fin = st_fin.append(st)
  bpctg -= 10

st_fin

In [ ]:
# Generating Excel File
budname = 'budget_' + str(round(inp_budget/1000000,2)).replace('.','_')
popname = 'pop_' + str(round(pop_pctg * 100)) + 'pc_'
curtime = datetime.datetime.now()
ts = datetime.datetime.strftime(curtime, '_%Y%m%d_%H%M')

with pd.ExcelWriter(root_path + '/Results/' + 'Zappa_Cost_Simulation_' + city + '_' + budname + '_' + popname + ts +'.xlsx') as writer:
    st_fin.to_excel(writer, sheet_name='Summary', index=False)
finmessage = 'Output File Directory :' + os.getcwd() + '/Results/' + 'Zappa_Cost_Simulation_' + city + '_' + budname + '_' + popname + ts +'.xlsx'
print(finmessage)